<a href="https://colab.research.google.com/github/Chesta1/nlp/blob/main/Text_generation_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
data = pd.read_csv("/content/data_with_labels.csv")
data.head()

,Unnamed: 0,title,date,SubTitle,site,dominant_topic,topic_labels,cluster
0,0,mitigating ethical risks in generative ai stra...,2024-01-03 00:00:00.000,artificial intelligence ai has been around for...,DSC,6,AI and Models,4
1,1,dsc weekly january,2024-01-02 00:00:00.000,announcements top stories in depth,DSC,7,Comprehensive Guide,6
2,2,mastering iot data management for business suc...,2024-01-01 00:00:00.000,in today s tech driven landscape the prolifera...,DSC,6,AI and Models,4
3,3,generative ai business model disruption the ny...,2024-01-01 00:00:00.000,will be all about changing business models due...,DSC,6,AI and Models,4
4,4,genai synthesizing dna sequences with llm tech...,2024-01-01 00:00:00.000,when people talk about large language models t...,DSC,8,Coding and Implementation,0


In [4]:
# ! pip install transformers
# !pip install datasets
# ! pip install accelerate -U

In [5]:
from datasets import Dataset

In [6]:
  data2 = {
      "subtitle":[subtitle for subtitle in data['SubTitle']],
      "title":[title for title in data['title']]
  }

  dataset = Dataset.from_dict(data2)

In [7]:
from transformers import AutoTokenizer,AutoModelForCausalLM

In [8]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
tokenizer.pad_token= tokenizer.eos_token

In [10]:
# Tokenize all subtitles without padding or truncation to find the maximum length
tokenized_subtitles = tokenizer(data2['subtitle'], add_special_tokens=True, return_length=True)
max_length_subtitles = max(tokenized_subtitles['length'])
print("Maximum length of subtitles:", max_length_subtitles)


Maximum length of subtitles: 37


In [11]:
# Tokenize all titles without padding or truncation to find the maximum length
tokenized_titles = tokenizer(data2['title'], add_special_tokens=True, return_length=True)
max_length_titles = max(tokenized_titles['length'])
print("Maximum length of titles:", max_length_titles)

Maximum length of titles: 24


In [12]:
sample_input = data2['subtitle'][0]  # Substitute 0 with any index if you want to check another sample
sample_target = data2['title'][0]  # Similarly, pick the corresponding target

In [13]:
sample_input_tokens = tokenizer(sample_input, max_length=max_length_subtitles, truncation=True, padding="max_length", return_tensors="pt")
sample_target_tokens = tokenizer(sample_target, max_length=max_length_titles, truncation=True, padding="max_length", return_tensors="pt")


In [14]:
# Inspecting input tokens
print("Input Token IDs:", sample_input_tokens['input_ids'])
print("Input Attention Mask:", sample_input_tokens['attention_mask'])
print("Decoded Input Tokens:", tokenizer.decode(sample_input_tokens['input_ids'][0]))

# Inspecting target tokens (for understanding, though typically not needed for generation models)
print("\nTarget Token IDs:", sample_target_tokens['input_ids'])
print("Target Attention Mask:", sample_target_tokens['attention_mask'])
print("Decoded Target Tokens:", tokenizer.decode(sample_target_tokens['input_ids'][0]))


Input Token IDs: tensor([[  433,  9542,  4430,   257,    72,   468,   587,  1088,   329,   867,
          4647,   475,   783,   340,   468,  1716,   257, 14713,  4775,   772,
          1871,  1729,  6276,   661,   780,   286,   262,  1152,   876,   257,
            72,  1100,   517, 50256, 50256, 50256, 50256]])
Input Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])
Decoded Input Tokens: artificial intelligence ai has been around for many decades but now it has become a buzzword even among non technical people because of the generative ai read more<|endoftext|><|endoftext|><|endoftext|><|endoftext|>

Target Token IDs: tensor([[ 2781, 29129, 15028,  7476,   287,  1152,   876,   257,    72, 10064,
           329,   257,  3338,   290,  5713,   257,    72,  3586, 50256, 50256,
         50256, 50256, 50256, 50256]])
Target Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [15]:
from transformers import DefaultDataCollator

def preprocess_data(examples):
    inputs = examples['subtitle']
    targets = examples['title']

    # Tokenize inputs with appropriate max_length for subtitles
    model_inputs = tokenizer(inputs, max_length=32, truncation=True, padding="max_length", return_tensors="pt")

    # Tokenize targets with appropriate max_length for titles, ensuring return_tensors is set to match inputs
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=32, truncation=True, padding="max_length", return_tensors="pt")['input_ids']

    # Ensure labels are tensors to match input formats - since return_tensors="pt" is used, they already are
    model_inputs['labels'] = labels

    return {key: val.detach().cpu().numpy() for key, val in model_inputs.items()}


tokenized_dataset = dataset.map(preprocess_data,batched=True)
data_collator = DefaultDataCollator()

Map:   0%|          | 0/449 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
from transformers import TrainingArguments,Trainer

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
# Split the dataset into train and test sets
train_testvalid = tokenized_dataset.train_test_split(test_size=0.2)
train_data = train_testvalid['train']
test_data = train_testvalid['test']

In [19]:
 # finetune the model
training_args = TrainingArguments(output_dir= "./results",
                                    num_train_epochs=5,
                                    per_device_train_batch_size=1,
                                    per_device_eval_batch_size=1,
                                    eval_accumulation_steps=4,
                                    evaluation_strategy="epoch",
                                    save_strategy="epoch",
                                    load_best_model_at_end=True,
                                    metric_for_best_model="eval_loss",
                                    logging_dir="./logs",)
trainer = Trainer(model=model,
                    args = training_args,
                    train_dataset=train_data,
                    eval_dataset=test_data,
                    tokenizer=tokenizer,
                    data_collator = data_collator,)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.855424
2,2.989600,2.814250
3,2.493500,2.873062
4,2.493500,2.920686
5,2.351400,2.990401


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=1795, training_loss=2.542474484244429, metrics={'train_runtime': 209.2907, 'train_samples_per_second': 8.577, 'train_steps_per_second': 8.577, 'total_flos': 29313699840000.0, 'train_loss': 2.542474484244429, 'epoch': 5.0})

In [20]:
from transformers import pipeline

In [21]:
generator = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, device=0)  # Add `device=0` if using GPU

new_subtitle="Machine and Deep Learning"

In [22]:
# Now, let's generate a title with adjusted settings
generated_titles = generator(new_subtitle, max_length=10, num_return_sequences=10,
                             temperature=0.85, top_k=85, top_p=0.70)

generated_title = generated_titles[0]["generated_text"]
print(f"Generated title: {generated_title}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated title: Machine and Deep Learning with the data in


In [23]:
# Encode the input text
input_ids = tokenizer.encode(new_subtitle, return_tensors='pt').to('cuda:0')

# Generate text directly using the model
output_ids = model.generate(input_ids, max_length=20, num_return_sequences=10,
                            temperature=0.90, top_k=90, top_p=0.95)

# Decode the generated ids to text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(f"Generated text: {generated_text}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: Machine and Deep Learning analysis
